# Challenge 3: Hidden Cold Joke
Using Python, call Github API to find out the cold joke contained in the 24 secret files in the following repo:

https://github.com/ironhack-datalabs/scavenger

The filenames of the secret files contain .scavengerhunt and they are scattered in different directories of this repo. The secret files are named from .0001.scavengerhunt to .0024.scavengerhunt. They are scattered randomly throughout this repo. You need to search for these files by calling the Github API, not searching the local files on your computer.

In [1]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import re
import sys

In [2]:
#Credentials
path = '/Users/josematos/Documents/github_credentials'
sys.path.insert(0, path)
from credentials import username, token

In [3]:
#Importing url contents and transform it to DataFrame
url = 'https://api.github.com/repos/ironhack-datalabs/scavenger/contents'
login = requests.get(url, auth=(username,token))
results = login.json()
data = pd.DataFrame(results)

In [4]:
#getting list of URL
name_lst = [col for col in data['path']]
url_lst = ['https://api.github.com/repos/ironhack-datalabs/scavenger/contents/'+name for name in name_lst]
del url_lst[0]

In [5]:
#Opening each folder in the Scavenger Repo
all_files = []
for url in url_lst:
    url1 = url
    login1 = requests.get(url1, auth=(username,token)).json()
    results = pd.DataFrame(login1)
    all_files.append(results)    
files_data = pd.concat(all_files)

In [6]:
#filtering the data
scavenger_data =  files_data[(files_data['size']>0)]

In [7]:
#Organizing the Data to sort it by something meaningful
sort_by_name = scavenger_data.sort_values('name')

In [8]:
# Getting the list of URL to open the files
scavenger_lst = [x for x in sort_by_name['download_url']]

In [9]:
# Opening Each file
lst_words = []
for a in scavenger_lst:
    login2 = requests.get(a, auth=(username,token))
    soup =BeautifulSoup(login2.text,'html.parser')
    lst_words.append(str(soup))
lst_words = [re.compile('[\W_]+').sub('',text) for text in lst_words] #Cleaning the list
#Creating sentecne
sentence = ' '.join(lst_words)
sentence

'In data science 80 percent of time spent is preparing data 20 percent of time is spent complaining about the need to prepare data'

In [10]:
# a = 'https://raw.githubusercontent.com/ironhack-datalabs/scavenger/master/98750/.0001.scavengerhunt'
# login2 = requests.get(a, auth=(username,token))
# soup =BeautifulSoup(login2.text,'html.parser')
# soup

In [11]:
# url1 = 'https://api.github.com/repos/ironhack-datalabs/scavenger/contents/15024'
# login1 = requests.get(url1, auth=(username,token))

# results1 = login1.json()
# results1
# data1 = pd.DataFrame(results1)
# data1
